# SSH and SSHFS
**31 Oct 2017**

Marco Tazzari

## 1. Using the SSH `config` file

It is a configuration file that lives in your home:
```bash
~/.ssh/config
```
If you have no `config` file in that directory, you can create it.

The `config` file:

- contains a sort of **address book** of the hosts you want to connect to.
- is very handy to connect to remote servers quickly and without too much typing.
- is portable: you can just copy your `config` file to another `~/.ssh/` and it works straight away 

Add an entry to the `config` file for each computer (*host*) you want to connect to. Each entry has to contain the following information:

```bash
host label
	Hostname NAME or IP ADDRESS   # e.g. computer.ast.cam.ac.uk or 123.11.22.33
	User USERNAME                 # your username on the host
	ForwardAgent yes              # to forward your SSH identity (i.e. the SSH Key)
	ForwardX11 yes                # to forward X11 window (equivalent to ssh -Y)
	UseKeychain yes               # for MacOS 12.2+ to remember ssh key pass phrases
	ForwardX11Trusted yes         # for MacOS to solve compatibility issues of X11
```
where `label` is just a label that you can freely choose for this connection.

Note that the indentation matters and the keywords have to be spelled with the correct upper cases.

Once you save the ssh config, you can connect to the computer as:
```bash
ssh label
```

### Connecting to a host directly

If you want to connect to a computer that you can directly reach (e.g. a gateway reachable from the WWW), you will just need an entry like:

```bash
host gateway
	Hostname 111.22.33.55          # IP address or name
	User john
	ForwardAgent yes
	ForwardX11 yes
	UseKeychain yes
	ForwardX11Trusted yes
```
where you have to enter the correct `Hostname` and your `User` name.

Once you saved the `config` file, you can connect to it with:
```bash
ssh gateway
```
Also all the other commands that rely on ssh will work likewise. E.g., copying a file is done with:
```bash
scp gateway:/path/to/file.txt .
```

NOTE: For the IoA you can find information on the gateways in the intranet pages **Computing -> Gateway and Remotely Accessible Machines**

### Connecting to a host behind a gateway

If you want to connect to a computer that is behind a gateway, you will just need two entries in the `config` files: one for the gateway and one for the desired computer. 

Having us already defined the gateway `gateway` in the entry above, we just need the entry for the desired computer:

```bash
host computer
	Hostname 192.168.0.30          # IP address or name
	User john
	ForwardAgent yes
	ProxyCommand ssh gateway -W %h:%p
	ForwardX11 yes
	ForwardX11Trusted yes
	UseKeychain yes   
```
where you have to enter the correct `Hostname` and your `User` name.

Once you saved the `config` file, you can connect to `computer` as simply as:
```bash
ssh computer
```

We notice that the tunneling magic is done by the additional line:
```bash
	ProxyCommand ssh <Gateway Hostname> -W %h:%p
```
where `<Gateway Hostname>` must be equal to the `host` label in the `config` file. I

Even if you pass through a gateway, you can copy files to and from `computer` as simply as:
```bash
scp computer:/path/to/file.txt .
```

## 2. Locally mount remote drives with `sshfs`

Sometimes it might be handy to locally mount a remote drive, so that the local Operative System recognises the remote drive (or directory) as local. 

On a Mac, you will need to install **Fuse for OS X** and **SSHFS**. The installers of both the tools can  be downloaded at the development page https://osxfuse.github.io/

On Linux, I believe `sshfs` should be already installed.

### Basic usage
---

#### Mounting
Mounting a remote drive locally can be done with:
```bash
sshfs TARGET LOCAL_DIR
```
`<TARGET>`: an ssh-reachable address, e.g. `server1:/path/to/my/home/`

`<LOCAL_DIR>`: a local directory on your computer, e.g. `~/remote_home/`

Example:

1) create a directory on your computer where you want to mount the drive, e.g.:
```bash
mkdir ~/remote_data/
```
2) mount the drive
```bash
sshfs computer:/path/ ~/remote_data/
```
You can now navigate in the remote directory `/path/` on `computer` by simply navigating locally in `~/remote_data/`.

Even more nicely, on a Mac you can drag-and-drop on `~/remote_data/` to move files to-from your local computer and the remote drive.

**Warning**: although in principle one could mount a whole machine or a remote drive, e.g.:
```bash
sshfs computer:/ ~/remote_data/
```
in my experience it is not good practice since it occupies more bandwidth and usually slows down the responsivity of the internet connection. 

#### Un-mounting
To un-mount, simply use the command `umount`:
```bash
umount TARGET
```
To unmount in the example above:
```
umount computer:/path/
```

Finally, I find it handy to define aliases in my `.bashrc` (or `.profile`, or `.bash_profile` file):
```bash
alias m_computer_path="sshfs computer:/path/ ~/remote_data/"
alias u_computer_path="umount computer:/path/"
```
so that I can mount the remote drive by simply typing in the terminal:
```bash
m_computer_path
```
and un-mount with:
```bash
u_computer_path
```